In [1]:
# Reset and Restart 

import sys
import os
import numpy as np
import time
from imp import reload
import threading
import csv
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO
from L4 import Focus
from L4.image_util import ImageSaver
reload(Focus)


reload(DisplayWindows)

#CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()

ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

%matplotlib qt

ce_system.inlet_rgb.turn_on_channel('B')
ce_system.inlet_rgb.turn_off_channel('R')



new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE
C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py
daq
newwave
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x000001FF1DE34988>, 'xy_stage': <L2.XYControl.PriorXY object at 0x000001FF37AF6B08>, 'objective': <L2.ZControl.PriorZ object at 0x000001FF37BBAA48>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x000001FF37BBAAC8>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x000001FF37BBA9C8>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x000001FF37BBABC8>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x000001FF37BBAA08>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x000001FF37BBAD08>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x000001FF37BAE808>, 'lysis_laser': <L2.LaserControl.NewWaveBNC object at 0x000001FF37BC1408>, 'excitation_wheel': <L2.FilterWheelControl.LumencorFilter object at 0x000001FF37BC1648>, 'filt

'R:Off, G:Off, B:On'

# Automation Objects

In [158]:
# Set up the Automation Method & Template 
#TEMPLATE_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\EclipseCellTemplate.txt"
#METHOD_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Brae\Cell.txt"

TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300CellTemplate.txt"
METHOD_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\Brae\Peak.txt"

from L4 import AutomatedControl
from L4 import Trajectory
reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)
auto_program.add_method(METHOD_PATH)
auto_program.repetitions=5

WHASSUZ ['inject\tw_d7\t21 cm\t5 mm\t0\t0\t0\t3 s\t\t0', 'SF\tw_e1\t24 cm\t5 mm\t0\t0\t20 kV\t300 s\t\t1', 'inject\tw_d6\t21 cm\t5 mm\t0\t0\t0\t3 s\t\t0', 'S1P\tw_e1\t24 cm\t5 mm\t0\t0\t20 kV\t300 s\t\t1', 'inject\tw_d5\t21 cm\t5 mm\t0\t0\t0\t3 s\t\t0', 'Mix\tw_e1\t24 cm\t5 mm\t0\t0\t20 kV\t300 s\t\t1']
ADDING
ADDING
ADDING
ADDING
ADDING
ADDING


In [60]:
# Start the Method
if not auto_program.continue_event.is_set():
    auto_program.continue_event.set()
else:
    #auto_program.start_run()
    pass

In [160]:
# Stop the Method
auto_program.stop_run()

ERROR:root:Error while moving during System Move. Aborting
ERROR:root:Exiting the run...


In [2]:
#ce_system.inlet_z.homing()ll

In [159]:
auto_program.start_run()

Exception in thread Thread-102132:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveTimeoutException: Move failed to complete in time
   at Thorlabs.MotionControl.GenericMotorCLI.GenericMotorCLI.Wait(Int32 waitTimeout)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveTo(Decimal position, Int32 waitTimeout)



After special True
Time to wait is  3
After special True
Time to wait is  300
After special True
Time to wait is  3


After special True
Time to wait is  300
After special True
Time to wait is  3


After special True
Time to wait is  300


In [16]:
ce_system.objective.set_z(0)

In [5]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): w_e0
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


In [7]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()
ce_system.detector.start()

In [4]:
ce_system.camera.set_binning(4)

In [11]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
cam.scalar=1
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


In [49]:
ce_system.xy_stage.controller.open()


# Set the Focus Plane for the cells

In [13]:
focus_plane = Focus.PlaneFocus(ce_system)

focus_plane.get_three_points()


In [30]:
# run this three times
focus_plane.give_response()

In [31]:
focus_plane.find_a_plane()

True

In [32]:
with open('focus_save_me','w') as fout:
    fout.write(f'focus_plane,{focus_plane._plane_coefficients}')

In [14]:
with open('focus_save_me','r') as fin:
    focus_plane._plane_coefficients= eval(fin.readline().split('focus_plane,')[1])
    

In [15]:
focus_plane.get_plane_focus()

In [16]:
focus_search = Focus.FindFocus(ce_system)

# Set the Capillary Adjustments

In [40]:
# Move the objective to the height and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(.05)

In [35]:
ce_system.inlet_z.set_rel_z(-0.01)


In [43]:
ce_system.inlet_z.set_rel_z(+1.5)

In [20]:
ce_system.inlet_z.set_rel_z(-.005)

In [39]:
ce_system.inlet_z.set_z(5)

In [36]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height

In [37]:
with open('save_me','w') as fout:
    fout.write(f'cap_difference,{cap_difference}')

In [17]:
with open('save_me','r') as fin:
    cap_difference= float(fin.readline().split(',')[1])
    

In [38]:
cap_difference

-1.1498000000000002

In [18]:
# PARAMETERS
target_x = 260*2
target_y = 79*2
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 500
_old_exp=100
_old_bin = 1
_old_channel = 0
bins = 1
conversion =0.0003125
obj_40x = 0.0001607717041800643


def pre_fluoresence(auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    if auto_shutter:
        ce_system.excitation_shutter.set_auto_on()
    #_old_exp = ce_system.camera.exposure
    #ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence(auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    #ce_system.camera.set_binning(_old_bin)
    ce_system.excitation_shutter.set_auto_off()
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)
    ce_system.camera.continuous_snap()

def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

def get_brightest_blobs(image):
    close_size=5
    open_size=5
    watershed_footprint = (5,5)
    # Make sure types are the same
    input_image = img_as_float(img)

    # Filter Image
    filtered_image = filters.median(input_image, behavior='ndimage')

    # Edge Detection
    edge_sobel = filters.sobel(filtered_image)

    # Threshold
    thresh = filters.threshold_otsu(filtered_image)
    binary_otsu = filtered_image > thresh

    # Binary Morphology Operations
    structure_element = morphology.disk(close_size)
    closed_image = morphology.binary_closing(binary_otsu, structure_element)
    structure_element = morphology.disk(open_size)
    opened_image = morphology.binary_opening(closed_image, structure_element)

    # Watershed
    distance = distance_transform_edt(opened_image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
    markers = label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=opened_image)
    
    # Labeling and Sorting
    regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))
    df = pd.DataFrame(regions)
    df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)
    
    return df

def move_to_blob(df):
    if df.shape[0] > 0:
        y=df['centroid-0'][0]*bins
        x=df['centroid-1'][0]*bins

        movex = (x-target_x)*conversion
        movey = (y-target_y)*conversion
        ce_system.xy_stage.set_rel_x(movex)
        ce_system.xy_stage.set_rel_y(-movey)
        
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    ce_system.lysis_laser.laser_fire()
    ce_system.objective.set_rel_z(-0.010)

    while time.time()-start_time < injection_time:
        if time.time()-start_time < 3:
            ce_system.objective.set_rel_z(0.005)
            ce_system.xy_stage.set_rel_x(0.001)
            ce_system.lysis_laser.laser_fire()
        time.sleep(0.35)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)

In [11]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [28]:
data_dir = r"C:\Users\Luke\Documents\Brae\9-19-2020\images"
fluor_saver = ImageSaver(ce_system, folder_prefix='Picking', data_folder=data_dir)
focus_saver = ImageSaver(ce_system, folder_prefix='Focusing', data_folder=data_dir)
lysis_saver = ImageSaver(ce_system, folder_prefix='Lysis', data_folder=data_dir)

def automatic_sampling():
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    time.sleep(3)
    
    # Test Alec's Focus and get data
    pre_fluoresence()
    focus_saver.add_callback()
    focus_search = Focus.FindFocus(ce_system)
    focus_search.search_step_global()
    focus_saver.remove_callback()
    post_fluoresence()
    focus_saver.save_image()
    
    # We don't Trust Alec's function yet
    fluor_channel = 3
    focus_plane.get_plane_focus()
    
    # Get Fluoresence Image for Best chance
    fluor_channel = 1
    pre_fluoresence()
    fluor_saver.add_callback()
    img = ce_system.camera.snap()
    fluor_saver.remove_callback()
    post_fluoresence()
    
    
    # Move to the brighest blob
    blobs = get_brightest_blobs(img)
    move_to_blob(blobs)
    
    
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    lysis_saver.add_callback()
    time.sleep(0.5)
    
    # on reruns, you just need to run this command when you have the cells in focus
    resp= True
    if resp:
        obj_height = ce_system.objective.read_z()
        print(obj_height)
        cap_z = get_cap_height(obj_height, cap_difference)
        z = ce_system.inlet_z.read_z()
        rel_z = cap_z-z
        rel_z += 0.02
        ce_system.inlet_z.set_rel_z(rel_z)
    else:
        print("Bring cells into focus.")
        raise RuntimeError("Cells not in Focus")

    time.sleep(5)
    
    # Start the Single Cell Injection
    limited_fire(ce_system, gravity_drop, voltage_level, injection_time)
    
    # Prepare to start the method
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(3)
    lysis_saver.remove_callback()
    lysis_saver.save_image()
    
    # Start the Method
    ce_system.objective.set_rel_z(-3)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()

NameError: name 'ImageSaver' is not defined

# TE300 Automation

In [124]:
 # PARAMETERS
target_x = 871
target_y = 585
rgb_chnl = 'B'
fluor_channel = 4
wait = 0.25
exp = 500
_old_exp=10
_old_bin = 2
_old_channel = 1
light_channel = 'cyan'
bins = 8
obj_60x = 0.0001927525057825752
conversion = obj_60x

def te300_presnap(ce_sys=ce_system, intensity=1):
    ce_system.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def te300_postsnap(ce_sys=ce_system, intensity=0):
    ce_system.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def pre_fluoresence(auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    ce_system.excitation_wheel.set_channel([light_channel])
    ce_system.camera.set_exposure(exp)
    #_old_exp = ce_system.camera.exposure
    ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence(auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    #ce_system.camera.set_binning(_old_bin)
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)
    ce_system.excitation_wheel.set_channel([])
    ce_system.camera.set_binning(_old_bin)
    ce_system.camera.continuous_snap()
    
def move_to_blob(df):
    if df.shape[0] > 0:
        y=df['centroid-0'][0]*bins
        x=df['centroid-1'][0]*bins

        movex = (x-target_x)*conversion
        movey = (y-target_y)*conversion
        ce_system.xy_stage.set_rel_x(movex)
        ce_system.xy_stage.set_rel_y(-movey)
        
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time, delta_z=0.002, delta_x=0.001, delta_time=0.25, under_pulses=2):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    
    ce_system.lysis_laser.laser_fire()
    ce_system.objective.set_rel_z(-delta_z*under_pulses)

    while time.time()-start_time < injection_time:
        if time.time()-start_time < 3:
            ce_system.objective.set_rel_z(delta_z)
            ce_system.xy_stage.set_rel_x(delta_x)
            ce_system.lysis_laser.laser_fire()
        time.sleep(delta_time)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)

ce_system.camera._presnap_callbacks=[]
ce_system.camera._postsnap_callbacks=[]
ce_system.camera.add_presnap_callback(te300_presnap,ce_sys=ce_system)
ce_system.camera.add_postsnap_callback(te300_postsnap, ce_sys=ce_system)

In [ ]:
data_dir = r"C:\Users\NikonTE300CE\Desktop\Brae\9-26-2020\images"
fluor_saver = ImageSaver(ce_system, folder_prefix='Picking', data_folder=data_dir)
focus_saver = ImageSaver(ce_system, folder_prefix='Focusing', data_folder=data_dir)
lysis_saver = ImageSaver(ce_system, folder_prefix='Lysis', data_folder=data_dir)

gravity_drop = 0
votlage_level = 3
injection_time = 5
delta_z = 0.002
delta_x = 0.001
delta_time = 0.2
under_pulses

def automatic_sampling():
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    time.sleep(3)
    
    # Get Fluoresence Image for Best chance
    pre_fluoresence()
    fluor_saver.add_callback()
    img = ce_system.camera.snap()
    fluor_saver.remove_callback()
    post_fluoresence()
    fluor_saver.save_image(timestamp=True, image_prefix='Bright_Cells', unique_folder=False)
    
    # Move to the brighest blob
    blobs = get_brightest_blobs(img)
    move_to_blob(blobs)
    
    # Bring the Cells into focus
    focus_plane.get_plane_focus()
    lysis_saver.add_callback()
    time.sleep(0.5)
    
    #Bring the capillary Down
    resp= True
    if resp:
        obj_height = ce_system.objective.read_z()
        print(obj_height)
        cap_z = get_cap_height(obj_height, cap_difference)
        z = ce_system.inlet_z.read_z()
        rel_z = cap_z-z
        ce_system.inlet_z.set_rel_z(rel_z)
    else:
        print("Bring cells into focus.")
        raise RuntimeError("Cells not in Focus")
    
    ce_system.inlet_z.wait_for_target(z+rel_z)
    time.sleep(0.5)
    # Start the Single Cell Injection
    limited_fire(ce_system, gravity_drop, voltage_level, injection_time)
    
    # Prepare to start the method
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(3)
    lysis_saver.remove_callback()
    lysis_saver.save_image()
    
    # Start the Method
    ce_system.objective.set_rel_z(-3)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()

In [59]:
las = ce_system.lysis_laser

In [60]:
las.laser_standby()

In [72]:
las.laser_fire()

# Find the Brightest Cell

In [104]:
fluor_channel=4
light_channel='cyan'
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()
fig, ax = plt.subplots()
ax.imshow(img)

HEYYOO


In [15]:
ce_system.filter_wheel.set_channel(1)

In [12]:
ce_system.outlet_pressure.rinse_pressure()

In [11]:
ce_system.outlet_pressure.release()

In [16]:
ce_system.inlet_rgb.turn_on_channel('B')

'R:On, G:Off, B:On'

In [17]:
ce_system.inlet_rgb.turn_off_channel('B')

'R:Off, G:Off, B:On'

In [162]:
fluor_channel=5
ligh_channel='uv'
pre_fluoresence()
focus_search.search_step_global()
post_fluoresence()

HEYYOO


In [103]:
post_fluoresence()

HEYYOO


In [81]:
fluor_channel=5
pre_fluoresence()

In [83]:
ce_system.camera.snap()

array([[327, 315, 316, ..., 229, 224, 234],
       [310, 299, 314, ..., 223, 222, 223],
       [308, 306, 303, ..., 224, 218, 217],
       ...,
       [210, 217, 204, ..., 204, 211, 195],
       [215, 206, 216, ..., 199, 203, 209],
       [210, 214, 209, ..., 199, 208, 196]], dtype=uint16)

In [102]:
ce_system.camera.stop()

True

In [13]:
ce_system.excitation_wheel.set_channel(['red'])

In [94]:
ce_system.camera.continuous_snap()

HEYYOO


True

In [271]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [236]:
# Search for the best focal plane near here
pre_fluoresence()
focus_search.search_step_global()
post_fluoresence()

HEYYOO


In [258]:
fluor_channel = 1
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()


HEYYOO


In [184]:
plt.figure()
plt.imshow(img)

In [259]:
close_size=5
open_size=5
watershed_footprint = (5,5)
# Make sure types are the same
input_image = img_as_float(img)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(filtered_image)
binary_otsu = filtered_image > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.figure()
plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

In [260]:
if df.shape[0] > 0:
    y=df['centroid-0'][0]*bins
    x=df['centroid-1'][0]*bins
    
    movex = (x-target_x)*conversion
    movey = (y-target_y)*conversion
    ce_system.xy_stage.set_rel_x(movex)
    ce_system.xy_stage.set_rel_y(-movey)

# Single Cell Injection/Load 

In [144]:
focus_plane.get_plane_focus()

In [19]:
ce_system.objective.set_rel_z(6)

In [406]:
#xy = ce_system.xy_stage.read_xy()
#ce_system.xy_stage.set_xy(xy)

In [423]:
fluor_channel=4
light_channel='cyan'
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()

fig, ax = plt.subplots()
ax.imshow(img)

HEYYOO


In [145]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    print(obj_height)
    cap_z = get_cap_height(obj_height, cap_difference)
    z = ce_system.inlet_z.read_z()
    rel_z = cap_z-z
    ce_system.inlet_z.set_rel_z(rel_z)
else:
    print("Bring cells into focus.")

are the cells in focus? (You may crush the capillary if not...) y
-0.0809


In [84]:
# Raise the objective for optimal plasma formation
ce_system.objective.set_rel_z(+.005)

In [147]:
# Lower the capillary (ADJUSTMENT)
ce_system.inlet_z.set_rel_z(+0.02)

# LYSE THE CELL (BELOW)

In [149]:
# Start the Single Cell Injection

# General Injection Parameters
injection_time = 3 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 10 # in kilovolts
delay = 0 # inseconds
arg = (5,0.001, 0.003,0.05, 0)
import threading

def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.outlet_pressure.release()
    ce_system.high_voltage.start()
    start_time = time.time()
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()
    
    first=False
    while time.time()-start_time < injection_time:
        time.sleep(0.1)
        if time.time()-start_time >10 and first:
            ce_system.lysis_laser.laser_fire()
            first=False
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    ce_system.outlet_pressure.seal()


threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()
#threading.Thread(target=fire_me, args=arg).start()

Fire
Finished 3.035632848739624


In [53]:
ce_system.lysis_laser.laser_fire()

In [193]:
las.laser_standby()

In [79]:
ce_system.outlet_pressure.rinse_pressure()

In [80]:
ce_system.outlet_pressure.release()

In [127]:
ce_system.inlet_z.set_z(3)

In [89]:
ce_system.inlet_z.set_rel_z(-3)

In [289]:
def fire_me(reps, delta_y, delta_z, delta_t,jump=0.02, under=0):
    
    
    las = ce_system.lysis_laser
    las.laser_standby()
    #ce_system.objective.set_rel_z(jump)
    las.laser_fire()
    time.sleep(delta_z)
    # Move down by under
    ce_system.objective.set_rel_z(-delta_z*under)
    
    for i in range(reps):
        las.laser_fire()
        ce_system.objective.set_rel_z(delta_z)
        ce_system.xy_stage.set_rel_y(delta_y)
        time.sleep(delta_t)
    
arg = (10,0.001, 0.002,0.1, 0.015)
threading.Thread(target=fire_me, args=arg).start()

In [73]:
ce_system.inlet_z.set_rel_z(1.2)

In [244]:
resp = ""
while resp != 'q':
    resp=input("Press enter to fire, press q to exit")
    if resp != 'q':
        ce_system.lysis_laser.laser_fire()

Press enter to fire, press q to exitq


In [109]:
auto_program.start_run()


In [150]:
# Start the Method
def go_ahead():
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(2)
    ce_system.objective.set_rel_z(-6)
    if not auto_program.continue_event.is_set():
        
        auto_program.continue_event.set()
    else:
        auto_program.start_run()
threading.Thread(target=go_ahead).start()

After special True
Time to wait is  3
After special True
Time to wait is  200


After special True
Time to wait is  15


After special True
Time to wait is  45


In [131]:
ce_system.outlet_pressure.rinse_pressure()

In [132]:
ce_system.outlet_pressure.release()

In [23]:
im = ImageSaver(ce_system,folder_prefix='degradation',data_folder=r"C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020")

In [89]:
ce_system.xy_stage.controller.open()

In [91]:
ce_system.outlet_pressure.controller.open()

In [13]:
im.remove_callback()

In [19]:
im = ImageSaver(ce_system,folder_prefix='degradation',data_folder=r"C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020")
def run_me():
    
    pre_fluoresence()
    im.add_callback()
    for i in range(100):
        img = ce_system.camera.snap()
        #fig, ax = plt.subplots()
        #ax.imshow(img)
    im.remove_callback()
    post_fluoresence()
    
threading.Thread(target=run_me).start()


HEYYOO


In [20]:
im.save_image(unique_folder=False)

C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00000.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00001.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00002.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00003.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_lau

C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00059.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00060.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00061.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00062.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00063.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykern

# PUT YOUR CRAP FUNCTIONS HERE

In [161]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [94]:
ce_system.outlet_z.homing()

In [34]:

ce_system.inlet_rgb.turn_on_channel('B')
ce_system.inlet_rgb.turn_off_channel('R')

'R:Off, G:Off, B:On'

In [162]:
# Stop Rinse 
ce_system.outlet_pressure.seal()

In [281]:
ce_system.objective.set_rel_z(-3)

In [8]:
ce_system.outlet_pressure.release()

In [18]:
ce_system.filter_wheel.set_channel(1)

In [87]:
ce_system.high_voltage.stop()

In [78]:
laser = ce_system.lysis_laser

In [292]:
laser.laser_standby()

In [291]:
laser.laser_stop()

In [293]:
laser.laser_fire()

In [11]:
ce_system.objective.set_rel_z(-3)

In [302]:
ce_system.excitation_wheel.set_channel(['green'])
ce_system.excitation_wheel.set_intensity(['green'],0)

In [127]:
ce_system.inlet_rgb.turn_on_channel('R')


'R:On, G:Off, B:Off'

In [126]:
ce_system.inlet_rgb.turn_off_channel('R')


'R:Off, G:Off, B:Off'

In [58]:
xy = ce_system.xy_stage.read_xy()
xy

[28.52200042501092, -1.8983000282868743]

In [68]:
ce_system.xy_stage.set_xy(xy)

In [19]:
ce_system.xy_stage.set_rel_x(0.05)

In [20]:
191.5-61.8

129.7

In [21]:
129.7*2

259.4

In [22]:
0.05/259.4

0.0001927525057825752

In [10]:
ce_system.inlet_z.set_z(10)

In [147]:
ce_system.outlet_pressure.seal()

In [6]:
cam = ce_system.camera

In [7]:
ce_system.inlet_z.controller.open()

In [9]:
ce_system.inlet_z.startup()

In [3]:
output = r"C:\Users\Luke\Documents\Data\images"
im_sav = ImageSaver(ce_system, data_folder=output, prefix='testing')

In [4]:
im_sav.add_callback()

In [7]:
cam.continuous_snap()

HEYYOO


True

In [8]:
cam.stop()

True

In [9]:
im_sav.remove_callback()

In [14]:
im_sav.save_image(unique_folder=False)

C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00000.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00001.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00002.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00003.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\i